# CloudMart Resource Tagging Cost Governance Analysis
## Week 10 Activity - Exploratory Data Analysis

**Objective:** Analyze cloud resource tagging compliance and cost visibility for CloudMart Inc.

**Task Sets Covered:**
- Task Set 1: Data Exploration
- Task Set 2: Cost Visibility
- Task Set 3: Tagging Compliance

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

---
## Task Set 1: Data Exploration
---

### Task 1.1: Load the dataset and display first 5 rows

In [2]:
# Load the dataset
# Use na_values parameter to treat empty strings as NaN
df = pd.read_csv('cloudmart_multi_account.csv', na_values=['', ' '])

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

Dataset loaded successfully!
Shape: (72, 12)

First 5 rows:


,AccountID,ResourceID,Service,Region,Department,Project,Environment,Owner,CostCenter,CreatedBy,MonthlyCostUSD,Tagged
0,1001,i-001,EC2,us-east-1,Marketing,CampaignApp,Prod,j.smith@cloudmart.com,CC101,Terraform,120,Yes
1,1001,i-002,EC2,us-east-1,Marketing,CampaignApp,Dev,NaN,CC101,Terraform,80,No
2,1001,s3-001,S3,us-east-1,Marketing,AdsAPI,Prod,j.smith@cloudmart.com,CC101,Jenkins,60,Yes
3,1001,s3-002,S3,us-east-1,Marketing,AdsAPI,Dev,NaN,CC101,Manual,25,No
4,1001,api-001,APIGateway,us-east-2,Marketing,AdsAPI,Prod,j.smith@cloudmart.com,CC101,Jenkins,70,Yes


In [3]:
# # Check for and remove duplicates
# initial_rows = len(df)
# df = df.drop_duplicates()
# duplicates_removed = initial_rows - len(df)

# print(f"Initial rows: {initial_rows}")
# print(f"Duplicates removed: {duplicates_removed}")
# print(f"Final rows: {len(df)}")

In [4]:
# Display dataset info
print("Dataset Information:")
df.info()

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   AccountID       72 non-null     int64 
 1   ResourceID      72 non-null     object
 2   Service         72 non-null     object
 3   Region          72 non-null     object
 4   Department      68 non-null     object
 5   Project         68 non-null     object
 6   Environment     68 non-null     object
 7   Owner           34 non-null     object
 8   CostCenter      68 non-null     object
 9   CreatedBy       72 non-null     object
 10  MonthlyCostUSD  72 non-null     int64 
 11  Tagged          72 non-null     object
dtypes: int64(2), object(10)
memory usage: 6.9+ KB


### Task 1.2: Check for missing values

In [5]:
# Check missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage (%)': missing_percentage
})

print("Missing Values Analysis:")
missing_df

Missing Values Analysis:


,Missing Count,Percentage (%)
AccountID,0,0.00
ResourceID,0,0.00
Service,0,0.00
Region,0,0.00
Department,4,5.56
Project,4,5.56
Environment,4,5.56
Owner,38,52.78
CostCenter,4,5.56
CreatedBy,0,0.00


### Task 1.3: Identify columns with most missing values

In [6]:
# Columns with missing values sorted
missing_sorted = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

print("Columns with Most Missing Values:")
print(missing_sorted)

# Visualize missing values
if len(missing_sorted) > 0:
    fig = px.bar(
        x=missing_sorted.index,
        y=missing_sorted['Percentage (%)'],
        title='Missing Values by Column (%)',
        labels={'x': 'Column', 'y': 'Missing Percentage (%)'},
        color=missing_sorted['Percentage (%)'],
        color_continuous_scale='Reds'
    )
    fig.show()

Columns with Most Missing Values:
             Missing Count  Percentage (%)
Owner                   38           52.78
Department               4            5.56
Project                  4            5.56
Environment              4            5.56
CostCenter               4            5.56


### Task 1.4: Count tagged vs untagged resources

In [7]:
# Count tagged vs untagged
tagged_counts = df['Tagged'].value_counts()
total_resources = len(df)

print("Tagged vs Untagged Resources:")
print(tagged_counts)
print(f"\nTotal Resources: {total_resources}")

# Create a simple visualization
fig = px.pie(
    values=tagged_counts.values,
    names=tagged_counts.index,
    title='Tagged vs Untagged Resources Distribution',
    color=tagged_counts.index,
    color_discrete_map={'Yes': '#2ecc71', 'No': '#e74c3c'}
)
fig.update_traces(textposition='inside', textinfo='percent+label+value')
fig.show()

Tagged vs Untagged Resources:
Tagged
No     38
Yes    34
Name: count, dtype: int64

Total Resources: 72


### Task 1.5: Calculate percentage of untagged resources

In [8]:
# Calculate untagged percentage
untagged_count = tagged_counts.get('No', 0)
tagged_count = tagged_counts.get('Yes', 0)
untagged_percentage = (untagged_count / total_resources) * 100
tagged_percentage = (tagged_count / total_resources) * 100

print(f"Percentage of Untagged Resources: {untagged_percentage:.2f}%")
print(f"Percentage of Tagged Resources: {tagged_percentage:.2f}%")
print(f"\n{'='*60}")
print(f"Tagged: {tagged_count} resources ({tagged_percentage:.2f}%)")
print(f"Untagged: {untagged_count} resources ({untagged_percentage:.2f}%)")

Percentage of Untagged Resources: 52.78%
Percentage of Tagged Resources: 47.22%

Tagged: 34 resources (47.22%)
Untagged: 38 resources (52.78%)


---
## Task Set 2: Cost Visibility
---

### Task 2.1: Calculate total cost of tagged vs untagged resources

In [9]:
# Cost by tagging status
cost_by_tagged = df.groupby('Tagged')['MonthlyCostUSD'].sum()

print("Total Cost by Tagging Status:")
print(cost_by_tagged)
print(f"\nTotal Monthly Cost: ${cost_by_tagged.sum():,.2f}")

# Visualize
fig = px.bar(
    x=cost_by_tagged.index,
    y=cost_by_tagged.values,
    title='Total Cost: Tagged vs Untagged Resources',
    labels={'x': 'Tagging Status', 'y': 'Monthly Cost (USD)'},
    color=cost_by_tagged.index,
    color_discrete_map={'Yes': '#2ecc71', 'No': '#e74c3c'},
    text=cost_by_tagged.values
)
fig.update_traces(texttemplate='$%{text:,.2f}', textposition='outside')
fig.show()

Total Cost by Tagging Status:
Tagged
No     2740
Yes    4680
Name: MonthlyCostUSD, dtype: int64

Total Monthly Cost: $7,420.00


### Task 2.2: Compute percentage of total cost that is untagged

In [10]:
# Calculate cost percentages
total_cost = df['MonthlyCostUSD'].sum()
untagged_cost = cost_by_tagged.get('No', 0)
tagged_cost = cost_by_tagged.get('Yes', 0)

untagged_cost_percentage = (untagged_cost / total_cost) * 100
tagged_cost_percentage = (tagged_cost / total_cost) * 100

print(f"Total Cost: ${total_cost:,.2f}")
print(f"\nTagged Resources Cost: ${tagged_cost:,.2f} ({tagged_cost_percentage:.2f}%)")
print(f"Untagged Resources Cost: ${untagged_cost:,.2f} ({untagged_cost_percentage:.2f}%)")
print(f"\nWARNING: {untagged_cost_percentage:.2f}% of total cost lacks proper tagging!")

Total Cost: $7,420.00

Tagged Resources Cost: $4,680.00 (63.07%)
Untagged Resources Cost: $2,740.00 (36.93%)



### Task 2.3: Identify department with most untagged cost

In [11]:
# Untagged cost by department
dept_untagged = df[df['Tagged'] == 'No'].groupby('Department')['MonthlyCostUSD'].sum().sort_values(ascending=False)

print("Untagged Cost by Department:")
print(dept_untagged)

if len(dept_untagged) > 0:
    print(f"\nDepartment with Most Untagged Cost: {dept_untagged.index[0]}")
    print(f"   Amount: ${dept_untagged.iloc[0]:,.2f}")

# Visualize
if len(dept_untagged) > 0:
    fig = px.bar(
        x=dept_untagged.index,
        y=dept_untagged.values,
        title='Untagged Cost by Department',
        labels={'x': 'Department', 'y': 'Untagged Cost (USD)'},
        color=dept_untagged.values,
        color_continuous_scale='Reds'
    )
    fig.show()

Untagged Cost by Department:
Department
Sales        480
Analytics    420
Finance      400
HR           340
DevOps       330
Marketing    280
Name: MonthlyCostUSD, dtype: int64

Department with Most Untagged Cost: Sales
   Amount: $480.00


### Task 2.4: Identify project with most cost overall

In [12]:
# Total cost by project
project_cost = df.groupby('Project')['MonthlyCostUSD'].sum().sort_values(ascending=False)

print("Total Cost by Project (Top 10):")
print(project_cost.head(10))

if len(project_cost) > 0:
    print(f"\nProject with Highest Cost: {project_cost.index[0]}")
    print(f"   Total Cost: ${project_cost.iloc[0]:,.2f}")

# Visualize top 10 projects
fig = px.bar(
    x=project_cost.head(10).values,
    y=project_cost.head(10).index,
    orientation='h',
    title='Top 10 Projects by Total Cost',
    labels={'x': 'Monthly Cost (USD)', 'y': 'Project'},
    color=project_cost.head(10).values,
    color_continuous_scale='Blues'
)
fig.show()

Total Cost by Project (Top 10):
Project
CRMTool          1920
LedgerDB          710
CI-CD             540
DataLake          540
CampaignApp       500
PayrollApp        490
DataWarehouse     460
AdsAPI            380
RecruitPortal     360
InfraMonitor      340
Name: MonthlyCostUSD, dtype: int64

Project with Highest Cost: CRMTool
   Total Cost: $1,920.00


### Task 2.5: Compare Prod vs Dev environments

In [13]:
# Environment analysis
env_analysis = df.groupby(['Environment', 'Tagged']).agg({
    'MonthlyCostUSD': 'sum',
    'ResourceID': 'count'
}).round(2)
env_analysis.columns = ['Total Cost (USD)', 'Resource Count']

print("Cost and Tagging by Environment:")
print(env_analysis)

# Environment summary
env_summary = df.groupby('Environment').agg({
    'MonthlyCostUSD': 'sum',
    'ResourceID': 'count',
    'Tagged': lambda x: (x == 'Yes').sum() / len(x) * 100
})
env_summary.columns = ['Total Cost (USD)', 'Resource Count', 'Tagging Rate (%)']
env_summary = env_summary.round(2)

print("\nEnvironment Summary:")
print(env_summary)

Cost and Tagging by Environment:
                    Total Cost (USD)  Resource Count
Environment Tagged                                  
Dev         No                  2000              28
Prod        Yes                 4680              34
Test        No                   250               6

Environment Summary:
             Total Cost (USD)  Resource Count  Tagging Rate (%)
Environment                                                    
Dev                      2000              28              0.00
Prod                     4680              34            100.00
Test                      250               6              0.00


In [14]:
# Visualize environment comparison
fig = go.Figure()

for env in df['Environment'].unique():
    env_data = df[df['Environment'] == env].groupby('Tagged')['MonthlyCostUSD'].sum()
    fig.add_trace(go.Bar(
        name=env,
        x=env_data.index,
        y=env_data.values,
        text=env_data.values,
        texttemplate='$%{text:,.0f}',
        textposition='auto'
    ))

fig.update_layout(
    title='Cost by Environment and Tagging Status',
    xaxis_title='Tagging Status',
    yaxis_title='Monthly Cost (USD)',
    barmode='group'
)
fig.show()

---
## Task Set 3: Tagging Compliance
---

### Task 3.1: Create Tag Completeness Score per resource

In [15]:
# Define tag fields
tag_fields = ['Department', 'Project', 'Environment', 'Owner', 'CostCenter']

# Calculate completeness score
df['TagCompletenessScore'] = df[tag_fields].notna().sum(axis=1)
df['TagCompletenessPercentage'] = (df['TagCompletenessScore'] / len(tag_fields)) * 100

print(f"Tag Completeness Score created (0-{len(tag_fields)} scale)")
print(f"\nTag Completeness Distribution:")
print(df['TagCompletenessScore'].value_counts().sort_index())

# Summary statistics
print(f"\nTag Completeness Statistics:")
print(f"  Mean Score: {df['TagCompletenessScore'].mean():.2f}/{len(tag_fields)}")
print(f"  Median Score: {df['TagCompletenessScore'].median():.2f}/{len(tag_fields)}")
print(f"  Min Score: {df['TagCompletenessScore'].min():.0f}/{len(tag_fields)}")
print(f"  Max Score: {df['TagCompletenessScore'].max():.0f}/{len(tag_fields)}")

Tag Completeness Score created (0-5 scale)

Tag Completeness Distribution:
TagCompletenessScore
0     4
4    34
5    34
Name: count, dtype: int64

Tag Completeness Statistics:
  Mean Score: 4.25/5
  Median Score: 4.00/5
  Min Score: 0/5
  Max Score: 5/5


In [16]:
# Visualize tag completeness distribution
completeness_dist = df['TagCompletenessScore'].value_counts().sort_index()

fig = px.bar(
    x=completeness_dist.index,
    y=completeness_dist.values,
    title='Tag Completeness Score Distribution',
    labels={'x': 'Completeness Score (0-5)', 'y': 'Number of Resources'},
    color=completeness_dist.index,
    color_continuous_scale='RdYlGn'
)
fig.show()

### Task 3.2: Find top 5 resources with lowest completeness scores

In [17]:
# Top 5 resources with lowest completeness
lowest_completeness = df.nsmallest(5, 'TagCompletenessScore')[[
    'ResourceID', 'Service', 'Department', 'Project', 'Environment',
    'Owner', 'CostCenter', 'TagCompletenessScore', 'TagCompletenessPercentage', 'MonthlyCostUSD'
]]

print("Top 5 Resources with Lowest Tag Completeness:")
lowest_completeness

Top 5 Resources with Lowest Tag Completeness:


,ResourceID,Service,Department,Project,Environment,Owner,CostCenter,TagCompletenessScore,TagCompletenessPercentage,MonthlyCostUSD
7,ebs-002,EBS,NaN,NaN,NaN,NaN,NaN,0,0.00,45
13,ec2-004,EC2,NaN,NaN,NaN,NaN,NaN,0,0.00,200
43,ebs-002,EBS,NaN,NaN,NaN,NaN,NaN,0,0.00,45
49,ec2-004,EC2,NaN,NaN,NaN,NaN,NaN,0,0.00,200
1,i-002,EC2,Marketing,CampaignApp,Dev,NaN,CC101,4,80.00,80


### Task 3.3: Identify most frequently missing tag fields

In [18]:
# Most frequently missing tag fields
missing_by_field = df[tag_fields].isnull().sum().sort_values(ascending=False)
missing_percentage_field = (missing_by_field / len(df) * 100).round(2)

missing_fields_df = pd.DataFrame({
    'Missing Count': missing_by_field,
    'Percentage (%)': missing_percentage_field
})

print("Most Frequently Missing Tag Fields:")
print(missing_fields_df)

# Visualize
fig = px.bar(
    x=missing_fields_df.index,
    y=missing_fields_df['Percentage (%)'],
    title='Missing Tag Fields (%)',
    labels={'x': 'Tag Field', 'y': 'Missing Percentage (%)'},
    color=missing_fields_df['Percentage (%)'],
    color_continuous_scale='Reds'
)
fig.show()

Most Frequently Missing Tag Fields:
             Missing Count  Percentage (%)
Owner                   38           52.78
Department               4            5.56
Project                  4            5.56
Environment              4            5.56
CostCenter               4            5.56


### Task 3.4: List all untagged resources and their costs

In [19]:
# Untagged resources
untagged_resources = df[df['Tagged'] == 'No'][[
    'ResourceID', 'Service', 'Region', 'Department', 'Project',
    'Environment', 'Owner', 'CostCenter', 'MonthlyCostUSD',
    'TagCompletenessScore', 'TagCompletenessPercentage'
]].sort_values('MonthlyCostUSD', ascending=False)

print(f"Total Untagged Resources: {len(untagged_resources)}")
print(f"Total Cost of Untagged Resources: ${untagged_resources['MonthlyCostUSD'].sum():,.2f}")
print(f"\nUntagged Resources (sorted by cost):")
untagged_resources

Total Untagged Resources: 38
Total Cost of Untagged Resources: $2,740.00

Untagged Resources (sorted by cost):


,ResourceID,Service,Region,Department,Project,Environment,Owner,CostCenter,MonthlyCostUSD,TagCompletenessScore,TagCompletenessPercentage
49,ec2-004,EC2,us-east-2,NaN,NaN,NaN,NaN,NaN,200,0,0.00
13,ec2-004,EC2,us-east-2,NaN,NaN,NaN,NaN,NaN,200,0,0.00
9,rds-002,RDS,us-west-2,Sales,CRMTool,Dev,NaN,CC102,150,4,80.00
45,rds-002,RDS,us-west-2,Sales,CRMTool,Dev,NaN,CC102,150,4,80.00
28,rds-004,RDS,us-west-1,Finance,LedgerDB,Dev,NaN,CC105,110,4,80.00
64,rds-004,RDS,us-west-1,Finance,LedgerDB,Dev,NaN,CC105,110,4,80.00
53,s3-004,S3,us-east-1,Analytics,DataLake,Dev,NaN,CC103,90,4,80.00
57,ecs-002,ECS,us-west-1,DevOps,CI-CD,Dev,NaN,CC104,90,4,80.00
21,ecs-002,ECS,us-west-1,DevOps,CI-CD,Dev,NaN,CC104,90,4,80.00
17,s3-004,S3,us-east-1,Analytics,DataLake,Dev,NaN,CC103,90,4,80.00


### Task 3.5: Export untagged resources to CSV

In [20]:
# Export untagged resources
output_file = 'untagged.csv'
untagged_resources.to_csv(output_file, index=False)

print(f"Untagged resources exported to: {output_file}")
print(f"   Total records exported: {len(untagged_resources)}")

Untagged resources exported to: untagged.csv
   Total records exported: 38


---
## Summary & Key Findings
---

In [21]:
# Generate comprehensive summary
print("="*80)
print("CLOUDMART RESOURCE TAGGING - EXECUTIVE SUMMARY")
print("="*80)

print(f"\nRESOURCE OVERVIEW")
print(f"   Total Resources: {len(df):,}")
print(f"   Tagged: {tagged_count:,} ({tagged_percentage:.1f}%)")
print(f"   Untagged: {untagged_count:,} ({untagged_percentage:.1f}%)")

print(f"\nCOST ANALYSIS")
print(f"   Total Monthly Cost: ${total_cost:,.2f}")
print(f"   Tagged Cost: ${tagged_cost:,.2f} ({tagged_cost_percentage:.1f}%)")
print(f"   Untagged Cost: ${untagged_cost:,.2f} ({untagged_cost_percentage:.1f}%)")

print(f"\nTAG COMPLIANCE")
print(f"   Average Completeness Score: {df['TagCompletenessScore'].mean():.2f}/5")
print(f"   Resources with Perfect Tags: {len(df[df['TagCompletenessScore'] == 5])}")
print(f"   Resources with No Tags: {len(df[df['TagCompletenessScore'] == 0])}")

print(f"\nKEY ISSUES")
if len(dept_untagged) > 0:
    print(f"   Department with Most Untagged Cost: {dept_untagged.index[0]} (${dept_untagged.iloc[0]:,.2f})")
if len(missing_by_field) > 0:
    print(f"   Most Missing Tag Field: {missing_by_field.index[0]} ({missing_percentage_field.iloc[0]:.1f}% missing)")
print(f"   High-Cost Untagged Resources: {len(untagged_resources[untagged_resources['MonthlyCostUSD'] > 100])}")

print(f"\nEXPORTED FILES")
print(f"   - untagged.csv (resources requiring remediation)")

print("\n" + "="*80)

CLOUDMART RESOURCE TAGGING - EXECUTIVE SUMMARY

RESOURCE OVERVIEW
   Total Resources: 72
   Tagged: 34 (47.2%)
   Untagged: 38 (52.8%)

COST ANALYSIS
   Total Monthly Cost: $7,420.00
   Tagged Cost: $4,680.00 (63.1%)
   Untagged Cost: $2,740.00 (36.9%)

TAG COMPLIANCE
   Average Completeness Score: 4.25/5
   Resources with Perfect Tags: 34
   Resources with No Tags: 4

KEY ISSUES
   Department with Most Untagged Cost: Sales ($480.00)
   Most Missing Tag Field: Owner (52.8% missing)
   High-Cost Untagged Resources: 6

EXPORTED FILES
   - untagged.csv (resources requiring remediation)



---
## Save Enhanced Dataset
---

In [22]:
# Save enhanced dataset with completeness scores
output_enhanced = 'cloudmart_with_scores.csv'
df.to_csv(output_enhanced, index=False)

print(f"Enhanced dataset saved to: {output_enhanced}")
print(f"   Includes Tag Completeness Score and Percentage columns")

Enhanced dataset saved to: cloudmart_with_scores.csv
   Includes Tag Completeness Score and Percentage columns


---
## Governance Report
---

In [23]:
# Generate Governance Report
from datetime import datetime

print("="*90)
print("CLOUDMART TAGGING GOVERNANCE REPORT")
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*90)

# 1. Percentage of Untagged Resources
print("\n1. UNTAGGED RESOURCES ANALYSIS")
print("-" * 90)
print(f"   Total Resources:              {total_resources}")
print(f"   Untagged Resources:           {untagged_count} ({untagged_percentage:.2f}%)")
print(f"   Tagged Resources:             {tagged_count} ({tagged_percentage:.2f}%)")
print(f"\n   STATUS: {untagged_percentage:.2f}% of resources lack proper tagging")

# 2. Total Untagged Cost
print("\n2. FINANCIAL IMPACT")
print("-" * 90)
print(f"   Total Monthly Cost:           ${total_cost:,.2f}")
print(f"   Untagged Resources Cost:      ${untagged_cost:,.2f} ({untagged_cost_percentage:.2f}%)")
print(f"   Tagged Resources Cost:        ${tagged_cost:,.2f} ({tagged_cost_percentage:.2f}%)")
print(f"\n   IMPACT: ${untagged_cost:,.2f}/month in spending lacks cost attribution")

# 3. Departments with Missing Tags
print("\n3. DEPARTMENTS WITH MISSING TAGS")
print("-" * 90)
dept_missing_analysis = df[df['Tagged'] == 'No'].groupby('Department').agg({
    'ResourceID': 'count',
    'MonthlyCostUSD': 'sum'
}).rename(columns={'ResourceID': 'Untagged Count', 'MonthlyCostUSD': 'Untagged Cost'})
dept_missing_analysis = dept_missing_analysis.sort_values('Untagged Cost', ascending=False)

for dept in dept_missing_analysis.index:
    count = dept_missing_analysis.loc[dept, 'Untagged Count']
    cost = dept_missing_analysis.loc[dept, 'Untagged Cost']
    print(f"   {dept:<20} {count:>3} resources    ${cost:>8,.2f}")

# 4. Recommendations for Governance Improvement
print("\n4. RECOMMENDATIONS FOR GOVERNANCE IMPROVEMENT")
print("-" * 90)

recommendations = []

# Recommendation 1: Based on untagged percentage
if untagged_percentage > 40:
    recommendations.append({
        'priority': 'HIGH',
        'area': 'Tagging Compliance',
        'recommendation': f'Implement mandatory tagging policy - {untagged_percentage:.1f}% of resources are untagged',
        'action': 'Enforce tag validation in IaC templates (Terraform, CloudFormation) to prevent untagged resource creation'
    })

# Recommendation 2: Based on missing Owner tag
owner_missing_pct = missing_percentage_field.loc['Owner']
if owner_missing_pct > 30:
    recommendations.append({
        'priority': 'HIGH',
        'area': 'Accountability',
        'recommendation': f'Owner tag is missing on {owner_missing_pct:.1f}% of resources',
        'action': 'Require Owner tag for all resources and implement automated email notifications to resource owners'
    })

# Recommendation 3: Based on environment-specific issues
dev_untagged_pct = (len(df[(df['Environment'] == 'Dev') & (df['Tagged'] == 'No')]) / len(df[df['Environment'] == 'Dev']) * 100) if len(df[df['Environment'] == 'Dev']) > 0 else 0
test_untagged_pct = (len(df[(df['Environment'] == 'Test') & (df['Tagged'] == 'No')]) / len(df[df['Environment'] == 'Test']) * 100) if len(df[df['Environment'] == 'Test']) > 0 else 0

if dev_untagged_pct == 100 or test_untagged_pct == 100:
    recommendations.append({
        'priority': 'MEDIUM',
        'area': 'Development Environments',
        'recommendation': 'Dev and Test environments have 0% tagging compliance',
        'action': 'Apply same tagging standards to non-production environments for complete cost visibility'
    })

# Recommendation 4: Based on high-cost untagged resources
high_cost_untagged = len(untagged_resources[untagged_resources['MonthlyCostUSD'] > 100])
if high_cost_untagged > 0:
    recommendations.append({
        'priority': 'HIGH',
        'area': 'Cost Management',
        'recommendation': f'{high_cost_untagged} high-cost resources (>$100/month) are untagged',
        'action': 'Prioritize tagging remediation for resources with monthly cost >$100'
    })

# Recommendation 5: Department-specific
if len(dept_untagged) > 0:
    top_dept = dept_untagged.index[0]
    top_dept_cost = dept_untagged.iloc[0]
    recommendations.append({
        'priority': 'MEDIUM',
        'area': 'Department Focus',
        'recommendation': f'{top_dept} department has highest untagged cost (${top_dept_cost:,.2f})',
        'action': f'Conduct tagging workshop with {top_dept} team and assign remediation tasks'
    })

# Recommendation 6: Automation
recommendations.append({
    'priority': 'MEDIUM',
    'area': 'Automation',
    'recommendation': 'Reduce manual resource creation to improve tagging compliance',
    'action': 'Migrate all Manual resource creation to Infrastructure-as-Code (current manual resources are 100% untagged)'
})

# Recommendation 7: Monitoring
recommendations.append({
    'priority': 'LOW',
    'area': 'Continuous Monitoring',
    'recommendation': 'Establish ongoing tagging compliance monitoring',
    'action': 'Set up weekly automated reports and dashboard for tracking tagging KPIs and compliance trends'
})

# Display recommendations
for i, rec in enumerate(recommendations, 1):
    print(f"\n   [{rec['priority']}] Recommendation #{i}: {rec['area']}")
    print(f"   Issue:  {rec['recommendation']}")
    print(f"   Action: {rec['action']}")

print("\n" + "="*90)
print("END OF REPORT")
print("="*90)

CLOUDMART TAGGING GOVERNANCE REPORT
Generated: 2025-11-12 23:18:01

1. UNTAGGED RESOURCES ANALYSIS
------------------------------------------------------------------------------------------
   Total Resources:              72
   Untagged Resources:           38 (52.78%)
   Tagged Resources:             34 (47.22%)

   STATUS: 52.78% of resources lack proper tagging

2. FINANCIAL IMPACT
------------------------------------------------------------------------------------------
   Total Monthly Cost:           $7,420.00
   Untagged Resources Cost:      $2,740.00 (36.93%)
   Tagged Resources Cost:        $4,680.00 (63.07%)

   IMPACT: $2,740.00/month in spending lacks cost attribution

3. DEPARTMENTS WITH MISSING TAGS
------------------------------------------------------------------------------------------
   Sales                  4 resources    $  480.00
   Analytics              6 resources    $  420.00
   Finance                6 resources    $  400.00
   HR                     6 reso

In [26]:
# Export Governance Report to PDF using ReportLab
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from datetime import datetime

# Create PDF document
output_pdf = 'cloudmart_governance_report.pdf'
doc = SimpleDocTemplate(output_pdf, pagesize=letter,
                        rightMargin=72, leftMargin=72,
                        topMargin=72, bottomMargin=18)

# Container for the 'Flowable' objects
elements = []

# Define styles
styles = getSampleStyleSheet()
title_style = ParagraphStyle(
    'CustomTitle',
    parent=styles['Heading1'],
    fontSize=18,
    textColor='#1f4788',
    spaceAfter=12,
    alignment=TA_CENTER
)
heading_style = ParagraphStyle(
    'CustomHeading',
    parent=styles['Heading2'],
    fontSize=14,
    textColor='#2c5aa0',
    spaceAfter=10,
    spaceBefore=12
)
normal_style = styles['Normal']
bold_style = ParagraphStyle(
    'Bold',
    parent=styles['Normal'],
    fontName='Helvetica-Bold'
)

# Title
elements.append(Paragraph('CloudMart Tagging Governance Report', title_style))
elements.append(Spacer(1, 0.3*inch))

# Section 1: Untagged Resources Analysis
elements.append(Paragraph('1. UNTAGGED RESOURCES ANALYSIS', heading_style))
elements.append(Paragraph(f'Total Resources: {total_resources}', normal_style))
elements.append(Paragraph(f'Untagged Resources: {untagged_count} ({untagged_percentage:.2f}%)', normal_style))
elements.append(Paragraph(f'Tagged Resources: {tagged_count} ({tagged_percentage:.2f}%)', normal_style))
elements.append(Spacer(1, 0.1*inch))
elements.append(Paragraph(f'<b>STATUS:</b> {untagged_percentage:.2f}% of resources lack proper tagging', bold_style))
elements.append(Spacer(1, 0.2*inch))

# Section 2: Financial Impact
elements.append(Paragraph('2. FINANCIAL IMPACT', heading_style))
elements.append(Paragraph(f'Total Monthly Cost: ${total_cost:,.2f}', normal_style))
elements.append(Paragraph(f'Untagged Resources Cost: ${untagged_cost:,.2f} ({untagged_cost_percentage:.2f}%)', normal_style))
elements.append(Paragraph(f'Tagged Resources Cost: ${tagged_cost:,.2f} ({tagged_cost_percentage:.2f}%)', normal_style))
elements.append(Spacer(1, 0.1*inch))
elements.append(Paragraph(f'<b>IMPACT:</b> ${untagged_cost:,.2f}/month in spending lacks cost attribution', bold_style))
elements.append(Spacer(1, 0.2*inch))

# Section 3: Departments with Missing Tags
elements.append(Paragraph('3. DEPARTMENTS WITH MISSING TAGS', heading_style))
dept_missing_analysis = df[df['Tagged'] == 'No'].groupby('Department').agg({
    'ResourceID': 'count',
    'MonthlyCostUSD': 'sum'
}).rename(columns={'ResourceID': 'Untagged Count', 'MonthlyCostUSD': 'Untagged Cost'})
dept_missing_analysis = dept_missing_analysis.sort_values('Untagged Cost', ascending=False)

for dept in dept_missing_analysis.index:
    count = dept_missing_analysis.loc[dept, 'Untagged Count']
    cost = dept_missing_analysis.loc[dept, 'Untagged Cost']
    elements.append(Paragraph(f'{dept}: {count} resources - ${cost:,.2f}', normal_style))
elements.append(Spacer(1, 0.2*inch))

# Section 4: Recommendations
elements.append(Paragraph('4. RECOMMENDATIONS FOR GOVERNANCE IMPROVEMENT', heading_style))
elements.append(Spacer(1, 0.1*inch))

recommendations = []

# Generate recommendations (same logic as original)
if untagged_percentage > 40:
    recommendations.append({
        'priority': 'HIGH',
        'area': 'Tagging Compliance',
        'recommendation': f'Implement mandatory tagging policy - {untagged_percentage:.1f}% of resources are untagged',
        'action': 'Enforce tag validation in IaC templates to prevent untagged resource creation'
    })

owner_missing_pct = missing_percentage_field.loc['Owner']
if owner_missing_pct > 30:
    recommendations.append({
        'priority': 'HIGH',
        'area': 'Accountability',
        'recommendation': f'Owner tag is missing on {owner_missing_pct:.1f}% of resources',
        'action': 'Require Owner tag for all resources with automated notifications'
    })

dev_untagged_pct = (len(df[(df['Environment'] == 'Dev') & (df['Tagged'] == 'No')]) / len(df[df['Environment'] == 'Dev']) * 100) if len(df[df['Environment'] == 'Dev']) > 0 else 0
test_untagged_pct = (len(df[(df['Environment'] == 'Test') & (df['Tagged'] == 'No')]) / len(df[df['Environment'] == 'Test']) * 100) if len(df[df['Environment'] == 'Test']) > 0 else 0

if dev_untagged_pct == 100 or test_untagged_pct == 100:
    recommendations.append({
        'priority': 'MEDIUM',
        'area': 'Development Environments',
        'recommendation': 'Dev and Test environments have 0% tagging compliance',
        'action': 'Apply same tagging standards to non-production environments'
    })

high_cost_untagged = len(untagged_resources[untagged_resources['MonthlyCostUSD'] > 100])
if high_cost_untagged > 0:
    recommendations.append({
        'priority': 'HIGH',
        'area': 'Cost Management',
        'recommendation': f'{high_cost_untagged} high-cost resources (>$100/month) are untagged',
        'action': 'Prioritize tagging remediation for resources with monthly cost >$100'
    })

if len(dept_untagged) > 0:
    top_dept = dept_untagged.index[0]
    top_dept_cost = dept_untagged.iloc[0]
    recommendations.append({
        'priority': 'MEDIUM',
        'area': 'Department Focus',
        'recommendation': f'{top_dept} department has highest untagged cost (${top_dept_cost:,.2f})',
        'action': f'Conduct tagging workshop with {top_dept} team'
    })

recommendations.append({
    'priority': 'MEDIUM',
    'area': 'Automation',
    'recommendation': 'Reduce manual resource creation to improve tagging compliance',
    'action': 'Migrate all Manual resource creation to Infrastructure-as-Code'
})

recommendations.append({
    'priority': 'LOW',
    'area': 'Continuous Monitoring',
    'recommendation': 'Establish ongoing tagging compliance monitoring',
    'action': 'Set up weekly automated reports and dashboard for tracking tagging KPIs'
})

# Add recommendations to PDF
rec_heading_style = ParagraphStyle(
    'RecHeading',
    parent=styles['Normal'],
    fontSize=12,
    fontName='Helvetica-Bold',
    spaceAfter=6
)

for i, rec in enumerate(recommendations, 1):
    elements.append(Paragraph(f"[{rec['priority']}] Recommendation #{i}: {rec['area']}", rec_heading_style))
    elements.append(Paragraph(f"<b>Issue:</b> {rec['recommendation']}", normal_style))
    elements.append(Paragraph(f"<b>Action:</b> {rec['action']}", normal_style))
    elements.append(Spacer(1, 0.15*inch))

# Build PDF
doc.build(elements)

print(f"Governance Report successfully exported to: {output_pdf}")
print(f"Total recommendations included: {len(recommendations)}")

Governance Report successfully exported to: cloudmart_governance_report.pdf
Total recommendations included: 7
